In [1]:
from pathlib import Path
import requests
import zipfile
import pandas as pd
import pyarrow.parquet as pq

def download_one_file_of_raw_data(year: int) -> Path:
    """"""
    URL = f'https://cdn.buenosaires.gob.ar/datosabiertos/datasets/transporte-y-obras-publicas/bicicletas-publicas/recorridos-realizados-{year}.zip'
    response= requests.get(URL) #, stream=True)

    
    if response.status_code == 200:
        
        path = f'../data/raw/recorridos-realizados-{year}.zip'
        open(path, "wb").write(response.content)     
        print(f'descargado año {year}')

        return path
    else:
        raise Exception(f'{URL} is not available')

In [2]:
import pyarrow as pa

def unzip_and_convert_csv_to_parquet(year: int) -> Path:
    nombre_archivo_zip = f"../data/raw/recorridos-realizados-{year}.zip"
        # Unzip the zip file
    with zipfile.ZipFile(nombre_archivo_zip, 'r') as archivo_zip:

        # Extract the CSV file from zip
        nombre_archivo_csv = archivo_zip.namelist()[0]  # 
        archivo_zip.extractall(f"../data/raw/")

        # Convert the DataFrame to Parquet dataframe
        df = pd.read_csv(f"../data/raw/{nombre_archivo_csv}", delimiter=',', decimal=".")

        
        nombre_archivo_parquet = f"rides_{year}.parquet"
        table = pa.Table.from_pandas(df)
        pq.write_table(table, f"../data/raw/{nombre_archivo_parquet}")

        path = f'../data/raw/rides_{year}.parquet'
    return path

In [27]:
download_one_file_of_raw_data(year=2022)

descargado año 2022


'../data/raw/recorridos-realizados-2022.zip'

In [3]:
unzip_and_convert_csv_to_parquet(year=2022)

'../data/raw/rides_2022.parquet'

In [5]:
import pandas as pd
rides = pd.read_parquet("../data/raw/rides_2022.parquet")

In [7]:
rides.head(10)

Unnamed: 0   X       Id_recorrido duracion_recorrido  \
0           1   1  13267975BAEcobici              2,610   
1           2   2  13268526BAEcobici                545   
2           3   3  13268400BAEcobici              2,061   
3           4   4  13268164BAEcobici             12,748   
4           5   5  13270010BAEcobici              4,337   
5           6   6  13269548BAEcobici              4,243   
6           7   7  13268959BAEcobici                932   
7           8   8  13267669BAEcobici              4,337   
8           9   9  13267592BAEcobici              3,735   
9          10  10  13376812BAEcobici                366   

  fecha_origen_recorrido id_estacion_origen    nombre_estacion_origen  \
0    2022-01-16 14:58:42         5BAEcobici        005 - Plaza Italia   
1    2022-01-16 17:26:27        51BAEcobici             051 - TUCUMAN   
2    2022-01-16 16:51:12       161BAEcobici           161 - Humahuaca   
3    2022-01-16 15:58:01       210BAEcobici     335 - General Urquiza   
4    2022-01-16 23:40:09       215BAEcobici           113 - Guatemala   
5    2022-01-16 20:42:30       268BAEcobici      399 - GARCIA DEL RIO   
6    2022-01-16 18:47:17       278BAEcobici              233 - MONROE   
7    2022-01-16 12:55:26       368BAEcobici           378 - AGRONOMIA   
8    2022-01-16 12:25:03       382BAEcobici            204 - Biarritz   
9    2022-01-29 15:13:01       433BAEcobici  273 - Plazoleta Colombia   

      direccion_estacion_origen  long_estacion_origen  lat_estacion_origen  \
0            Av. Sarmiento 2601            -58.420954           -34.580550   
1      Tucuman & 9 De Julio Av.            -58.382126           -34.601478   
2                3912 Humahuaca            -58.419676           -34.602078   
3  Figueroa Alcorta & Sarmiento            -58.411278           -34.572165   
4                Guatemala 4773            -58.424996           -34.585878   
5       Av. García del Río 3182            -58.477000           -34.550300   
6                   2519 Superi            -58.469813           -34.564122   
7               4351 San Martin            -58.482079           -34.598070   
8                 Biarritz 2403            -58.477390           -34.605470   
9                 1619 Brandsen            -58.373726           -34.637697   

  fecha_destino_recorrido id_estacion_destino        nombre_estacion_destino  \
0     2022-01-16 15:42:12        210BAEcobici          335 - General Urquiza   
1     2022-01-16 17:35:32        174BAEcobici  174 - MINISTERIO DE EDUCACION   
2     2022-01-16 17:25:33        117BAEcobici              117 - HUMBERTO 1°   
3     2022-01-16 19:30:29        382BAEcobici                 204 - Biarritz   
4     2022-01-17 00:52:26        205BAEcobici    125 - F.J.Santamaria de Oro   
5     2022-01-16 21:53:13        268BAEcobici           399 - GARCIA DEL RIO   
6     2022-01-16 19:02:49        236BAEcobici   254 - Plaza Rafael Hernandez   
7     2022-01-16 14:07:43        368BAEcobici                378 - AGRONOMIA   
8     2022-01-16 13:27:18          5BAEcobici             005 - Plaza Italia   
9     2022-01-29 15:19:07          6BAEcobici            006 - Parque Lezama   

              direccion_estacion_destino  long_estacion_destino  \
0           Figueroa Alcorta & Sarmiento             -58.411278   
1  Marcelo T. de Alvear & Rodriguez Peña             -58.391768   
2                             Peru  1016             -58.374176   
3                          Biarritz 2403             -58.477390   
4      F.J.Santamaria de Oro & Guatemala             -58.428016   
5                Av. García del Río 3182             -58.477000   
6                Vuelta de Obligado 2004             -58.455166   
7                        4351 San Martin             -58.482079   
8                     Av. Sarmiento 2601             -58.420954   
9             Avenida Martin Garcia, 295             -58.369758   

   lat_estacion_destino       id_usuario modelo_bicicleta  Género  
0  

In [8]:
rides.shape

(2922805, 19)

In [9]:
rides.id_estacion_origen.nunique()

331

In [12]:
#We keep only the columns that interest us and we rename them
rides = rides[["fecha_origen_recorrido","id_estacion_origen"]]
## Remove the "BAEcobici" part and convert to int type
rides['id_estacion_origen'] = rides['id_estacion_origen'].str.replace('BAEcobici', '').astype(int)
## Rename columns
rides.rename(columns={"fecha_origen_recorrido":"pickup_datetime","id_estacion_origen":"pickup_location_id"},inplace=True)

In [14]:
rides["pickup_datetime"] = pd.to_datetime(rides["pickup_datetime"], format='%Y-%m-%d %H:%M:%S')


In [15]:
rides.head(10)

pickup_datetime  pickup_location_id
0 2022-01-16 14:58:42                   5
1 2022-01-16 17:26:27                  51
2 2022-01-16 16:51:12                 161
3 2022-01-16 15:58:01                 210
4 2022-01-16 23:40:09                 215
5 2022-01-16 20:42:30                 268
6 2022-01-16 18:47:17                 278
7 2022-01-16 12:55:26                 368
8 2022-01-16 12:25:03                 382
9 2022-01-29 15:13:01                 433

In [16]:
rides.pickup_location_id.nunique()

331

In [17]:
rides["pickup_datetime"].describe()

count                          2922805
mean     2022-07-10 15:35:32.911203840
min                2022-01-01 00:11:07
25%                2022-04-06 20:58:08
50%                2022-07-20 11:28:05
75%                2022-10-11 21:37:32
max                2022-12-31 23:55:39
Name: pickup_datetime, dtype: object

In [19]:
rides = rides[rides.pickup_datetime >= '2022-01-01']
rides = rides[rides.pickup_datetime < '2023-01-01']
rides["pickup_datetime"].describe()

count                          2922805
mean     2022-07-10 15:35:32.911203840
min                2022-01-01 00:11:07
25%                2022-04-06 20:58:08
50%                2022-07-20 11:28:05
75%                2022-10-11 21:37:32
max                2022-12-31 23:55:39
Name: pickup_datetime, dtype: object

In [22]:
rides.to_parquet("../data/transformed/validated_rides_2022.parquet")